# Baseline v3 - Unsloth 최적화 버전

이 베이스라인 코드는 기존 v3 코드를 **Unsloth 라이브러리**를 사용하여 최적화한 버전입니다.

**주요 변경 사항:**
- **Unsloth 적용**: 더 적은 메모리로 2배 이상 빠른 파인튜닝을 수행합니다.
- **SFTTrainer 사용**: Transformers의 복잡한 수동 학습 루프를 `trl`의 `SFTTrainer`로 대체하여 코드를 간소화했습니다.
- **`FastVisionModel` 사용**: Unsloth의 Vision-Language 모델 로딩 클래스를 사용하여 모델 로딩 및 양자화 과정을 단순화했습니다.

Colab의 GPU 환경(T4 GPU)에서 개발되었습니다.
- **런타임 > 런타임 유형 변경 > T4 GPU**로 설정해주세요.

# 1. 환경 준비

Unsloth 및 최신 라이브러리를 설치합니다.

- 아래 셀 실행 후 **런타임 > 세션 다시 시작**을 반드시 진행해주세요.

In [ ]:
import os, re
import torch
if "COLAB_" in "".join(os.environ.keys()):
    print("Installing Unsloth for Google Colab...")
    v = re.match(r"[0-9\.]{3,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo -q
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer -q
    !pip install --no-deps unsloth -q
#    !pip install --no-deps --upgrade timm -q # For Gemma 3N

# 2. 데이터 준비

구글 드라이브를 마운트하고 대회 데이터를 압축 해제합니다.   
(*kaggle로 진행하는 교육생들은 `/kaggle/input/`에 있는 데이터셋으로 진행해주세요!)

- `train.csv`, `train/`
- `test.csv`, `test/`
- `sample_submission.csv`

In [ ]:
# 구글드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 데이터 압축 해제 (약 1분 소요)
!unzip -q "/content/drive/My Drive/251023/data.zip" -d "/content/"

# 3. 라이브러리, 데이터, 설정 로드

In [ ]:
import os, random
import pandas as pd
from PIL import Image
import torch
from datasets import Dataset, DatasetDict
from unsloth import FastVisionModel
from trl import SFTTrainer, SFTConfig
from unsloth.chat_templates import get_chat_template
from unsloth.trainer import UnslothVisionDataCollator
from tqdm import tqdm

# 이미지 로드 시 픽셀 제한 해제
Image.MAX_IMAGE_PIXELS = None

# 디바이스 GPU 우선 사용 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

# 설정값 정의
MODEL_ID = "unsloth/Qwen3-VL-8B-Instruct-unsloth-bnb-4bit" # Unsloth의 4bit 양자화 모델
MAX_SEQ_LENGTH = 2048
MAX_NEW_TOKENS = 8
SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# 데이터셋 로드
train_df = pd.read_csv("/content/train.csv")
test_df  = pd.read_csv("/content/test.csv")

# 학습 시간을 줄이기 위해 200개 샘플만 사용 (실제 대회에서는 전체 데이터를 사용하세요)
train_df = train_df.sample(n=200, random_state=SEED).reset_index(drop=True)

# 4. Unsloth 모델 및 LoRA 어댑터 로딩

Unsloth의 `FastVisionModel`을 사용하여 4bit 양자화된 모델과 토크나이저를 간편하게 로드합니다. 이후 LoRA 어댑터를 추가하여 학습 준비를 마칩니다.

In [ ]:
model, tokenizer = FastVisionModel.from_pretrained(
    model_name = MODEL_ID,
    max_seq_length = MAX_SEQ_LENGTH,
    dtype = None,
    load_in_4bit = True,
    use_gradient_checkpointing = "unsloth",
)

# LoRA 어댑터 추가
model = FastVisionModel.get_peft_model(
    model,
    finetune_vision_layers     = True,
    finetune_language_layers   = True,
    finetune_attention_modules = True,
    finetune_mlp_modules       = True,
    r = 16,
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    random_state = SEED,
    use_rslora = False,
    loftq_config = None,
)

model.print_trainable_parameters()

# 5. 프롬프트 템플릿 및 데이터 포맷팅

학습 및 추론에 사용할 프롬프트 템플릿을 정의하고, Hugging Face `datasets` 라이브러리를 사용하여 데이터를 모델의 Chat Template에 맞게 변환합니다. 이 과정은 기존의 Custom Dataset/Dataloader를 대체합니다.

In [ ]:
# 시스템 프롬포트도 커스터마이징 필수! (정확률 5%~15% 향상)
SYSTEM_INSTRUCT = (
    "You are a helpful visual question answering assistant. "
    "Answer using exactly one letter among a, b, c, or d. No explanation."
)

def build_mc_prompt(question, a, b, c, d):
    return (
        f"{question}\n"
        f"(a) {a}\n(b) {b}\n(c) {c}\n(d) {d}\n\n"
        "정답을 반드시 a, b, c, d 중 하나의 소문자 한 글자로만 출력하세요."
    )

# 학습 데이터를 모델의 대화 형식(messages)으로 변환하는 함수
def format_train_data(example):
    img = Image.open(example["path"]).convert("RGB")
    user_text = build_mc_prompt(str(example["question"]), str(example["a"]), str(example["b"]), str(example["c"]), str(example["d"]))
    gold = str(example["answer"]).strip().lower()

    messages = [
        {"role":"system","content":[{"type":"text","text":SYSTEM_INSTRUCT}]},
        {"role":"user","content":[{"type":"image","image":img},{"type":"text","text":user_text}]},
        {"role":"assistant","content":[{"type":"text","text":gold}]}
    ]
    return {"messages": messages}

# Qwen3 모델에 맞는 Chat Template 설정
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "qwen3-instruct",
)

# 최종적으로 Trainer가 사용할 'text' 필드를 만드는 함수
def formatting_prompts_func(examples):
    convos = examples["messages"]
    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in convos]
    return { "text" : texts }

# Pandas DataFrame을 Hugging Face Dataset으로 변환
raw_dataset = Dataset.from_pandas(train_df)

# 데이터 포맷팅 적용
formatted_dataset = raw_dataset.map(format_train_data, remove_columns=list(raw_dataset.features))
final_dataset = formatted_dataset.map(formatting_prompts_func, batched=True, remove_columns=list(formatted_dataset.features))

# 훈련/검증 데이터 분리
dataset_split = final_dataset.train_test_split(test_size=0.1, seed=SEED)
train_ds = dataset_split["train"]
valid_ds = dataset_split["test"]

print("Final training sample:\n", train_ds[0]['text'])

# 6. SFTTrainer를 사용한 파인튜닝

Unsloth와 `trl` 라이브러리의 `SFTTrainer`를 사용하여 모델 파인튜닝을 진행합니다. 이 방식은 기존의 수동 학습 루프보다 훨씬 간결하고 효율적입니다.

- 200개 샘플 학습 시 약 5~10분 소요됩니다.

In [ ]:
FastVisionModel.for_training(model) # 학습 모드 활성화

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_ds,
    eval_dataset = valid_ds,
    data_collator = UnslothVisionDataCollator(model, tokenizer),
    args = SFTConfig(
        per_device_train_batch_size = 1,
        per_device_eval_batch_size = 1,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60, # 데모용, 실제로는 num_train_epochs=1 등으로 설정
        # num_train_epochs = 1,
        learning_rate = 1e-4,
        logging_steps = 1,
        evaluation_strategy = "steps",
        eval_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = SEED,
        output_dir = "outputs",
        report_to = "none",
        remove_unused_columns = False,
        dataset_text_field = "",
        dataset_kwargs = {"skip_prepare_dataset": True},
        max_length = MAX_SEQ_LENGTH,
    ),
)

trainer.train()

# 7. 추론 및 제출 파일 생성

학습된 LoRA 어댑터를 사용하여 테스트 데이터에 대한 추론을 수행하고, `submission.csv` 파일을 생성합니다.

- 전체 테스트 데이터 추론 시 약 30분~1시간 소요됩니다.

In [ ]:
FastVisionModel.for_inference(model) # 추론 모드 활성화

# 모델의 응답에서 정답(a, b, c, d)만 추출하는 함수
def extract_choice(text: str) -> str:
    text = text.strip().lower()
    lines = [l.strip() for l in text.splitlines() if l.strip()]
    if not lines:
        return "a" # 응답이 없는 경우 'a'로 대체
    last = lines[-1]
    if last in ["a", "b", "c", "d"]:
        return last
    tokens = last.split()
    for tok in tokens:
        if tok in ["a", "b", "c", "d"]:
            return tok
    return "a" # 정답을 찾지 못한 경우 'a'로 대체

# 추론 루프
preds = []
for i in tqdm(range(len(test_df))):
    row = test_df.iloc[i]
    img = Image.open(row["path"]).convert("RGB")
    user_text = build_mc_prompt(row["question"], row["a"], row["b"], row["c"], row["d"])

    messages = [
        {"role":"system","content":[{"type":"text","text":SYSTEM_INSTRUCT}]},
        {"role":"user","content":[{"type":"image","image":img}, {"type":"text","text":user_text}]}
    ]

    # Chat Template 적용
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    # 토큰화 및 GPU로 이동
    inputs = tokenizer(text=[text], images=[img], return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=MAX_NEW_TOKENS, do_sample=False)

    decoded_output = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    preds.append(extract_choice(decoded_output))

# 제출 파일 생성
submission = pd.DataFrame({"id": test_df["id"], "answer": preds})
submission.to_csv("/content/submission.csv", index=False)
print("Saved /content/submission.csv")
print("Submission file sample:")
print(submission.head())